In [1]:
#util
import os
import time

#Math/arrays
import numpy as np
import pandas as pd

import optimizationFuncs as optim
import cpso
from importlib import reload

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 

#File io
#from Bio.PDB import *
from loadFile import *
from makePDB import *

#PSO
from threeDMaxGrad import *
from myUtils import *
# Import PySwarms
import pyswarms as ps
import numpy.random as random

%load_ext autoreload
%autoreload 2


from importlib import reload
from multiprocessing import Pool

In [2]:
#==========================================================================
# Make directory if it doesn't exist
#--------------------------------------------------------------------------
if not os.path.exists('my_folder'):
    os.makedirs('my_folder')

In [12]:
## constants
#inFile = "../../examples/GSDB/inputs/KR_1mb/chr20_matrix.txt"
#inFile = "../../examples/GSDB/inputs/KR_1mb/chr22_list.txt" #Works
#inFile = "../../examples/input/chr19.txt"# Works
#inFile = "../../examples/GSDB/inputs/KR_1mb/chr20_list.txt"
inFile = "../../examples/simulatedDataSet/Simulation/data/regular/regular90.txt"

## load file 
lstCons=loadFile.loadFileFunc(loadFile(), inFile)
 

square mat
Number of points :  4690
numBins  99.0
[[ 0.  1. 40.]
 [ 0. 70.  1.]
 [ 0. 69.  3.]
 ...
 [96. 99. 12.]
 [97. 98. 44.]
 [97. 99.  7.]]


In [17]:
lstCons[:,0] = lstCons[:,0].astype('int')
lstCons[:,1] = lstCons[:,1].astype('int')

scaleFactor = 25/max(lstCons[:,2])
lstCons[:,2] = lstCons[:,2]*scaleFactor
lstCons

array([[ 0.        ,  1.        , 18.18181818],
       [ 0.        , 70.        ,  0.45454545],
       [ 0.        , 69.        ,  1.36363636],
       ...,
       [96.        , 99.        ,  5.45454545],
       [97.        , 98.        , 20.        ],
       [97.        , 99.        ,  3.18181818]])

In [5]:
class outputObj:
    def __init__(self, xyzData, outputFile):
        self.xyzData = xyzData
        self.outputFile = outputFile
        
        #defaults
        self.recordName = []
        for i in range(len(xyzData)):
            self.recordName.append('ATOM')
            

    def myfunc(self):
        print("Hello my name is David")

In [6]:
def cost(xOyOz):
    cost = 0
    d = 0;
    structure = variables;
    

    #loop through existing data
    for k in range(len(lstCons)):
        print("yo2", lstCons)
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        
        if (IF <= 0) :
            continue
        

        if (abs(i - j) == 1 ) :
            IF = 1.0 * maxIF

        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );     

        z = str_dist - dist;

        # objective function       
        d += (z**2);     

        print(d)

        # cost
        cost = -(n/2) - (n*np.log(np.sqrt(d/n)));

        return cost

In [7]:
def opt_func(X):
    n_particles = X.shape[0]  # number of particles
    
    d = 0;
    #print(X)


    structure = variables #this is xyz
    allDists = np.zeros(n_particles)
    
    cost= 0
    for particle in range(n_particles):
        for i in range(len(structure)):
            structure[i][0]=X[particle][i]
            structure[i][1]=X[particle][i+len(structure)]
            structure[i][2]=X[particle][i+len(structure)*2]
            
        
        #loop through existing data
        for k in range(len(lstCons)):
            i = int(lstCons[k,0]);    
            j = int(lstCons[k,1]);    
            IF = lstCons[k,2];  
            dist = lstCons[k,3];
            if (IF <= 0) :
                continue


            if (abs(i - j) == 1 ) :
                IF = 1.0 * maxIF

            x1=structure[i][0];  x2=structure[j][0];
            y1=structure[i][1];  y2=structure[j][1];
            z1=structure[i][2];  z2=structure[j][2];

            str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );     

            z = str_dist - dist;

            # objective function  
            d += (z**2)
            
            #cost
            cost += -(n/2) - (n*np.log(np.sqrt(d/n)));
            
        
        allDists[particle] = cost
    
    return allDists

In [8]:
def getSpear():
    SUM = 0.0;

    Len = int(n * (n - 1) / 2) 
    Dist = np.zeros(Len);
    WishDist = np.zeros(Len);
    count = 1;
    structure = variables;
    for k in range(len(lstCons)-1):
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        # structure distance   
        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );
        SUM = SUM + ((str_dist - dist)**2);

        # calculate spearman_correlation and Pearson correlation
        if (i != j and IF > 0 and count < Len  ):   
            Dist[count] = str_dist;
            WishDist[count]= dist;
            count = count + 1

    SUM = SUM / len(lstCons);    
    rmse = np.sqrt(SUM);

    # let's convert to a dataframe
    df = pd.DataFrame({'Dist': Dist, 'wishDist': WishDist})
    pearsonCoeff = df.corr(method = 'pearson')
    spearmanCoeff = df.corr(method = 'spearman')
    print("preoptim  : ", spearmanCoeff.iloc[0]['wishDist']) 

In [11]:
## this is convert to distance
CONVERT_FACTOR_R = np.arange(0.1, 2, .2) # this is alpha and can be looped
#CONVERT_FACTOR_R=[0.9]
MAX_ITERATION = 500; # maximum number of iterations

AVG_DIST = 10.0  # an arbitrary distance

n=int(max(max(lstCons[:,0]),max(lstCons[:,1])))+1
print("n = ",n)

bestAlpha= CONVERT_FACTOR_R[0]
lstConsReset = lstCons
for CONVERT_FACTOR in CONVERT_FACTOR_R : 
    lstCons = lstConsReset
    ## Find the average IF
    avgIF = 0.0
    for i in range(len(lstCons)):
        avgIF = avgIF + float(lstCons[i][2])
    avgIF = avgIF/len(lstCons)
    
    maxIF = 0.0
    ## scale average distance to AVG_DIST
    avgDist = 10.0;
    avgAdjIF = 0.0;
    avgAdjCount = 0;
    totalIF = 0;

    for i in range(len(lstCons)):
        x = lstCons[i][0]
        y = lstCons[i][1]
        IF = lstCons[i][2]
        lstCons[i][2] = IF / avgIF  # normallize IF by avgIF
        IF = lstCons[i][2]
        dist = 1/(IF**CONVERT_FACTOR)
        avgDist = avgDist + dist

        totalIF = totalIF +  IF

        if ( IF > maxIF):
            maxIF = lstCons[i][2]
        # Find the adjacent position IF
        if ( abs(x-y)==1):
            avgAdjCount= avgAdjCount+1
            avgAdjIF =  avgAdjIF + IF

    avgDist = avgDist/len(lstCons)
    avgAdjIF = avgAdjIF/avgAdjCount

    maxIF = min(avgAdjIF, maxIF)

    ## TODO Add adjacent if none exist
    
    print('TODO Added missing adjacent constraint...')

    print('Number of constraints: = ', n)
    maxD = 0
    distResultsList= []
    for i in range(len(lstCons)):
        IF = lstCons[i,2];
        dist = AVG_DIST/ ((IF**CONVERT_FACTOR)* avgDist)
        distResultsList.append(dist)
        if (dist > maxD):
            maxD = dist;

    result = np.hstack((lstCons, np.atleast_2d(distResultsList).T))
    lstCons = result
    print('Max distance is: = \n', maxD); 
    
    ## Optimization
    ## Initialize Structure
    #=========================================================================
    
    smooth_factor = 1e-6 ; # for numerical stability
    LEARNING_RATE = 0.1; # Specify the learning rate
    NEAR_ZERO =0.00001; # used to signify a boundary of convergence
    NUM_FIGURES_OUTPUT=5

    thisStr =  [];
    R = [-0.5,0.5];
    for i in range(n):
        xyz = np.array([random.random(),random.random(),random.random()]) * (R[1]-R[0]) + min(R) 
        thisStr.append(xyz)
    
    ## Variables declaration
    #=========================================================================
    thislen = len(lstCons)

    Sum_Grad = np.zeros(n);
    variables = thisStr;
    oldobj = 0;
    getSpear()
    ######### PSO
    variables = np.array(variables)

    dist = 1/(IF**CONVERT_FACTOR)

    swarm_size = 300
    dim = n*3       # Dimension of X
    epsilon = 1
    options = {'c1': 0.5, 'c2':0.3, 'w':0.9}

    # Call an instance of PSO
    #optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
    #                                dimensions=dim,
    #                                options=options)
    
    options = {'c1': 0.5, 'c2':0.3, 'w':0.9, 'k': 3, 'p': 2}

    optimizer = ps.single.LocalBestPSO(n_particles=swarm_size,
                                    dimensions=dim,
                                    options=options )

    # Perform optimization
    cost, bestPXYZ = optimizer.optimize(opt_func, iters=100, n_processes=10)   
    for i in range(len(variables)):
        variables[i][0]=bestPXYZ[i]
        variables[i][1]=bestPXYZ[i+len(variables)]
        variables[i][2]=bestPXYZ[i+len(variables)*2]

    getSpear()

        
        
    

    #========================================================================
    # scoring using spearman correlation, pearson correlation and  RMSD     
    #------------------------------------------------------------------------
    # calculate rmse    
    SUM = 0.0;

    Len = int(n * (n - 1) / 2) 
    Dist = np.zeros(Len);
    WishDist = np.zeros(Len);
    count = 1;
    structure = variables;
    for k in range(len(lstCons)):
        i = int(lstCons[k,0]);    j = int(lstCons[k,1]);    IF = lstCons[k,2];  dist = lstCons[k,3];
        # structure distance   
        x1=structure[i][0];  x2=structure[j][0];
        y1=structure[i][1];  y2=structure[j][1];
        z1=structure[i][2];  z2=structure[j][2];

        str_dist = threeDMaxGrad.calEuclidianDist(threeDMaxGrad(), x1,y1,z1,x2,y2,z2 );
        SUM = SUM + ((str_dist - dist)**2);

        # calculate spearman_correlation and Pearson correlation
        if (i != j and IF > 0 and count<Len  ):   
            Dist[count] = str_dist;
            WishDist[count]= dist;
            count = count + 1

    SUM = SUM / len(lstCons);    
    rmse = np.sqrt(SUM);

    # let's convert to a dataframe
    df = pd.DataFrame({'Dist': Dist, 'wishDist': WishDist})
    pearsonCoeff = df.corr(method = 'pearson')
    spearmanCoeff = df.corr(method = 'spearman')
   
    if CONVERT_FACTOR == CONVERT_FACTOR_R[0]:#first run
        bestSpearmanRHO = spearmanCoeff.iloc[0]['wishDist']
        bestPearsonRHO = pearsonCoeff.iloc[0]['wishDist']
        bestMat =  variables
    elif bestSpearmanRHO < spearmanCoeff.iloc[0]['wishDist']:
        bestSpearmanRHO = spearmanCoeff.iloc[0]['wishDist']
        bestPearsonRHO = pearsonCoeff.iloc[0]['wishDist']
        bestMat =  variables
        bestAlpha = CONVERT_FACTOR

print("RMSE  : ", rmse)    
print("bestPearsonRHO  : ", bestPearsonRHO) 
print("bestSpearmanRHO  : ", bestSpearmanRHO) 
print("Best Alpha : ",bestAlpha)

2020-11-24 13:43:40,303 - pyswarms.single.local_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 3, 'p': 2}


n =  100
TODO Added missing adjacent constraint...
Number of constraints: =  100
Max distance is: = 
 11.359141953542258
preoptim  :  0.16926694481388718


pyswarms.single.local_best: 100%|██████████|100/100, best_cost=-8.3e+7
2020-11-24 13:46:33,958 - pyswarms.single.local_best - INFO - Optimization finished | best cost: -83023237.0050871, best pos: [ 1.01365952  0.72865411  0.79244255  0.04803357  0.37397084  0.33991835
  0.23749939  0.41546087  0.65214931  0.8764288   0.68215688  0.30884331
  0.95952875  0.70527947  0.71152355  1.11059452  0.03913697  0.21962884
  0.75564087  0.14119301  0.32809401  0.60946287  0.21411554  0.67681328
 -0.07781427  0.43812402  0.88191649  0.47803166  0.11805239  0.94700704
  0.16974364  0.19063135  0.18369334  0.0168471   0.2044638   0.83654489
  0.55581302 -0.03422718  0.74832543  0.2021656   0.38908687  1.02270035
  0.69220896  0.37880664  0.7217096   0.66575368  0.32903349 -0.7351837
  0.30363119  0.36464613  0.93176427  1.0538355   0.48477171  0.95303183
  0.08401227  0.88497837  0.85471904  0.64115572  0.41489152  0.63415292
  0.57376631  0.11104332  0.13615642 -0.1964427   0.88027175  0.80166427
 

preoptim  :  0.1453361513128939
TODO Added missing adjacent constraint...
Number of constraints: =  100
Max distance is: = 
 14.415241559751118
preoptim  :  0.1434367982685619


pyswarms.single.local_best: 100%|██████████|100/100, best_cost=-8.29e+7
2020-11-24 13:49:26,836 - pyswarms.single.local_best - INFO - Optimization finished | best cost: -82913172.56182562, best pos: [0.12020898 0.64362052 0.74882047 0.39518814 0.54010037 0.03019096
 0.96895631 0.70448592 0.93840064 0.44791802 0.22232405 0.73972989
 0.55832987 0.73604758 0.61135406 0.2873158  0.24692649 0.75532665
 0.46452508 0.40714402 0.99682415 0.3950135  0.65798214 0.00699797
 0.71514932 0.21675936 0.50694228 0.4541101  0.75452234 0.00707199
 0.28577145 0.21557072 0.60176085 0.40807987 0.19762725 0.68737409
 0.34702685 0.30270722 0.81621451 0.10223835 0.52972148 0.93731619
 0.16105032 0.96094134 0.95836979 0.44738775 0.73031947 0.28909619
 0.93822207 0.42241165 0.92708547 0.7789648  0.71690853 0.69058349
 0.34950545 0.16696931 0.53617904 0.77291003 0.27071154 0.08902804
 0.32524739 0.93105838 0.70611405 0.1901061  0.65466538 0.97147092
 0.05592858 0.20990368 0.37288644 0.81722621 0.20711783 0.409416

preoptim  :  0.15053003903564222
TODO Added missing adjacent constraint...
Number of constraints: =  100
Max distance is: = 
 17.819776861535072
preoptim  :  0.1341434287314741


pyswarms.single.local_best:  12%|█▏        |12/100, best_cost=-8.29e+7Process ForkPoolWorker-99:
Process ForkPoolWorker-94:
Process ForkPoolWorker-98:
Process ForkPoolWorker-100:
Process ForkPoolWorker-91:
Process ForkPoolWorker-97:
Process ForkPoolWorker-96:
Process ForkPoolWorker-93:
Process ForkPoolWorker-92:
Process ForkPoolWorker-95:
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()



KeyboardInterrupt: 

  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.

In [ ]:
#========================================================================
# create pdb
#------------------------------------------------------------------------
#Increase structure Size

xyz4pdb = myUtils.convert2xyz(myUtils(), n, bestMat) 
scale=100/np.amax(xyz4pdb)
xyz4pdb = xyz4pdb* scale
#output pdb file
outputData= np.round_(xyz4pdb ,3 )

outputFile = 'yolo.pdb' #output directory.


if os.path.exists(outputFile):
    os.remove(outputFile)

output = outputObj(outputData,outputFile)

pdbMaker = makePDB()
makePDB.mat2pdb(pdbMaker, output) # Converts the mat XYZ coordinates to PDB format.